# Redes Neurais

Nome: Ricardo Coutinho Cordeiro

Professor: Ulisses Brisolara Corrêa


##Importações básicas

In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score


#MLP utilizando sklearn e Validação Cruzada

Para começar a estudar validação cruzada e usar uma MLP de forma básica, você deverá utilizar o sklearn para implementar a rede neural, pois é uma maneira rápida e prática de aplicação.

Materiais de apoio:


*   https://scikit-learn.org/stable/modules/cross_validation.html

*   https://www.w3schools.com/python/python_ml_cross_validation.asp

*   https://inria.github.io/scikit-learn-mooc/python_scripts/02_numerical_pipeline_cross_validation.html

*   https://pieriantraining.com/understanding-cross-validation-in-scikit-learn-with-cross_validate/

MLP:
*   https://scikit-learn.org/stable/modules/neural_networks_supervised.html

## Baixar dataset

Para facilitar a aplicação das técnicas solicitadas, o dataset utilizado nessa etapa do trabalho será um dataset sobre vinhos, disponível na biblioteca sklearn.

Validação cruzada é uma técnica muito util, mas pode se tornar custosa caso o conjunto de dados utilizado seja muito grande ou caso seja utilizado um número muito grande de **folds**.

Mais informações sobre ele podem ser encontrados no link:
*   https://scikit-learn.org/stable/datasets/toy_dataset.html#wine-recognition-dataset

In [ ]:
from sklearn.datasets import load_wine

wine = load_wine()
dataset = pd.DataFrame(wine.data, columns = wine.feature_names)
dataset['target'] = wine.target
dataset.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


##Validação cruzada

In [ ]:
# pipeline com normalizacao e o modelo SVM
pipeline = Pipeline([
    ('normalizador', StandardScaler()),
    ('svm', SVC(kernel='linear'))
])

# StratifiedKFold para garantir que cada fold tenha a mesma proporcao de classes
skf = StratifiedKFold(n_splits=5)

# metricas a serem avaliadas
metricas = {
    'acuracia': make_scorer(accuracy_score),
    'precisao': make_scorer(precision_score, average='macro'),
    'recall': make_scorer(recall_score, average='macro'),
    'f1': make_scorer(f1_score, average='macro')
}

# validacao cruzada com multiplas metricas
pontuacoes = cross_val_score(pipeline, X, y, cv=skf, scoring='accuracy')
pontuacoes_precisao = cross_val_score(pipeline, X, y, cv=skf, scoring='precision_macro')
pontuacoes_recall = cross_val_score(pipeline, X, y, cv=skf, scoring='recall_macro')
pontuacoes_f1 = cross_val_score(pipeline, X, y, cv=skf, scoring='f1_macro')

# resultados
print(f"Acuracia: {pontuacoes.mean():.2f} ± {pontuacoes.std():.2f}")
print(f"Precisao: {pontuacoes_precisao.mean():.2f} ± {pontuacoes_precisao.std():.2f}")
print(f"Recall: {pontuacoes_recall.mean():.2f} ± {pontuacoes_recall.std():.2f}")
print(f"F1-score: {pontuacoes_f1.mean():.2f} ± {pontuacoes_f1.std():.2f}")


Acuracia: 0.96 ± 0.01
Precisao: 0.96 ± 0.02
Recall: 0.97 ± 0.01
F1-score: 0.96 ± 0.01


#MLP utilizando pytorch aplicado a imagens

Nesta etapa do trabalho, você deverá utilizar PyTorch para implementar uma rede MLP para a tarefa de classificação de dígitos no conjunto de dados MNIST.

O conjunto de caracteres manuscritos MNIST é frequentemente utilizado em tarefas introdutórias de inteligência artificial. Portanto, apesar de ser uma tarefa mais complexa, você provavelmente encontrará bastante material auxiliar disponível. Uma maneira de ter acesso ao conjunto de dados é utilizando a versão *built-in* disponível no pytorch, apresentada no ultimo link.

Materiais de apoio:

*   https://machinelearningmastery.com/building-multilayer-perceptron-models-in-pytorch/

*   https://hutsons-hacks.info/building-a-pytorch-binary-classification-multi-layer-perceptron-from-the-ground-up

*   https://pytorch.org/vision/main/datasets.html

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# transformacoes para os dados de treinamento e teste
transformacao = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# carrega o MNIST
conjunto_treinamento = datasets.MNIST(root='./data', train=True, download=True, transform=transformacao)
conjunto_teste = datasets.MNIST(root='./data', train=False, download=True, transform=transformacao)

# DataLoaders para treinamento e teste
carregador_treinamento = DataLoader(conjunto_treinamento, batch_size=64, shuffle=True)
carregador_teste = DataLoader(conjunto_teste, batch_size=1000, shuffle=False)

# definir a arquitetura da rede MLP
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28*28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        x = x.view(-1, 28*28)  # achatar as imagens em um vetor de 28*28
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

# inicializa o modelo, funcao de perda e otimizador
modelo = MLP()
criterio = nn.CrossEntropyLoss()
otimizador = optim.Adam(modelo.parameters(), lr=0.001)
agendador = optim.lr_scheduler.StepLR(otimizador, step_size=1, gamma=0.9)

# treinaa e avalia o modelo
def treinar_e_avaliar(modelo, carregador_treinamento, carregador_teste, criterio, otimizador, agendador, epocas=10):
    for epoca in range(1, epocas + 1):
        modelo.train()
        perda_acumulada = 0.0
        for indice_lote, (dados, alvos) in enumerate(carregador_treinamento):
            otimizador.zero_grad()
            saida = modelo(dados)
            perda = criterio(saida, alvos)
            perda.backward()
            otimizador.step()
            perda_acumulada += perda.item()

            if indice_lote % 100 == 0:
                print(f'Epoca [{epoca}/{epocas}], Lote [{indice_lote}/{len(carregador_treinamento)}], Perda: {perda.item():.4f}')

        agendador.step()

        modelo.eval()
        perda_teste = 0
        corretos = 0
        with torch.no_grad():
            for dados, alvos in carregador_teste:
                saida = modelo(dados)
                perda_teste += criterio(saida, alvos).item()
                previsao = saida.argmax(dim=1, keepdim=True)
                corretos += previsao.eq(alvos.view_as(previsao)).sum().item()

        perda_teste /= len(carregador_teste.dataset)
        acuracia = 100. * corretos / len(carregador_teste.dataset)
        print(f'\nConjunto de teste: Perda media: {perda_teste:.4f}, Acuracia: {corretos}/{len(carregador_teste.dataset)} ({acuracia:.0f}%)\n')

treinar_e_avaliar(modelo, carregador_treinamento, carregador_teste, criterio, otimizador, agendador, epocas=10)

Epoca [1/10], Lote [0/938], Perda: 2.3089
Epoca [1/10], Lote [100/938], Perda: 0.1276
Epoca [1/10], Lote [200/938], Perda: 0.2242
Epoca [1/10], Lote [300/938], Perda: 0.2252
Epoca [1/10], Lote [400/938], Perda: 0.2098
Epoca [1/10], Lote [500/938], Perda: 0.2452
Epoca [1/10], Lote [600/938], Perda: 0.2317
Epoca [1/10], Lote [700/938], Perda: 0.0875
Epoca [1/10], Lote [800/938], Perda: 0.0944
Epoca [1/10], Lote [900/938], Perda: 0.1889

Conjunto de teste: Perda media: 0.0001, Acuracia: 9666/10000 (97%)

Epoca [2/10], Lote [0/938], Perda: 0.1638
Epoca [2/10], Lote [100/938], Perda: 0.1085
Epoca [2/10], Lote [200/938], Perda: 0.1408
Epoca [2/10], Lote [300/938], Perda: 0.1784
Epoca [2/10], Lote [400/938], Perda: 0.0542
Epoca [2/10], Lote [500/938], Perda: 0.1306
Epoca [2/10], Lote [600/938], Perda: 0.1770
Epoca [2/10], Lote [700/938], Perda: 0.1662
Epoca [2/10], Lote [800/938], Perda: 0.1898
Epoca [2/10], Lote [900/938], Perda: 0.1423

Conjunto de teste: Perda media: 0.0001, Acuracia: 9758

#Dicas gerais para o desenvolvimento do trabalho:
*   Não esqueça de realizar a divisão entre conjuntos de treino e teste, e de se assegurar de que não haja vazamentos, como visto na tarefa da semana anterior.
*   A tarefa desta semana ainda é considerada uma tarefa de classificação, então o conteúdo sobre métricas de avaliação da tarefa anterior pode ser um guia útil ao avaliar a performance das redes MLP desenvolvidas neste trabalho.